# HW

1. Take 10 pictures of objects within the classifications of CIFAR-10 and change the image such that the format is the same (32x32 pixel resolution). Use your trained CNN model to classify the images. 
2. Train two more modified versions of the CNN model to improve the results of the first model. 

In [6]:
import keras.datasets.cifar10 as cifar10
from keras.optimizers import SGD, Adam
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, Conv3D, MaxPooling2D, MaxPooling3D, BatchNormalization
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from matplotlib.ticker import FormatStrFormatter
from matplotlib import pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report, ConfusionMatrixDisplay
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn import tree, datasets
from scipy.io import arff
from time import time
from os.path import join
from os import system, getcwd, startfile
from timeit import default_timer as timer
import tensorflow as tf
import math
import json
import inspect
import concurrent.futures as cf  # doesn't work with sklearn
import pandas as pd
import numpy as np
import copy as copy
import statistics as stt
import seaborn as sns
import pickle
import sys
sns.set_theme()
%matplotlib inline
tf.compat.v1.Session(
    config=tf.compat.v1.ConfigProto(log_device_placement=True))

(X_train, y_train), (X_test, y_test) = cifar10.load_data()
y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)


Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: NVIDIA GeForce GTX 1650 Ti, pci bus id: 0000:01:00.0, compute capability: 7.5



In [10]:
class CNN():
    def __init__(self, X_train, y_train, X_test, y_test, model=1):
        # build model
        if model == 1:
            self.model1()
        # train model
        epochs = 50
        self.model.fit(X_train, y_train, epochs=epochs, batch_size=500, verbose=1)
        self.model.summary()

    def model1(self):
        activation1 = 'relu'
        channel1 = 100
        channel2 = 200
        channel3 = 400
        model = Sequential()
        model.add(Conv2D(channel1, (9, 9),
                input_shape=X_train.shape[1:], activation=activation1))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Conv2D(channel2, (5, 5), activation=activation1))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Conv2D(channel3, (3, 3), activation=activation1))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Flatten())
        model.add(Dense(channel3, activation=activation1))
        model.add(Dropout(0.5))
        model.add(Dense(160, activation=activation1))
        model.add(Dense(40, activation=activation1))
        model.add(Dense(10, activation='softmax'))
        model.compile(loss='categorical_crossentropy',
                    optimizer='adam', metrics=['accuracy'])
        self.model = model

    def model2(self):
        pass

    def model3(self):
        pass

    def evaluate1(self):
        # top 1 accuracy
        pass

    def evaluate2(self):
        # top 3 accuracy
        pass

    def evaluate3(self):
        # top 5 accuracy
        pass




In [12]:
CNN(X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test, model=1)

Epoch 1/50
100/100 [==============================] - 6s 53ms/step - loss: 2.6050 - accuracy: 0.1511
Epoch 2/50
100/100 [==============================] - 5s 52ms/step - loss: 1.8628 - accuracy: 0.3132
Epoch 3/50
100/100 [==============================] - 5s 52ms/step - loss: 1.6503 - accuracy: 0.4009
Epoch 4/50
100/100 [==============================] - 5s 52ms/step - loss: 1.5554 - accuracy: 0.4397
Epoch 5/50
100/100 [==============================] - 5s 52ms/step - loss: 1.4632 - accuracy: 0.4748
Epoch 6/50
100/100 [==============================] - 5s 53ms/step - loss: 1.4049 - accuracy: 0.5012
Epoch 7/50
100/100 [==============================] - 5s 52ms/step - loss: 1.3378 - accuracy: 0.5245
Epoch 8/50
100/100 [==============================] - 5s 52ms/step - loss: 1.2769 - accuracy: 0.5446
Epoch 9/50
100/100 [==============================] - 5s 53ms/step - loss: 1.2323 - accuracy: 0.5629
Epoch 10/50
100/100 [==============================] - 5s 53ms/step - loss: 1.1695 - accura